In [6]:
import re
from nltk.tokenize import word_tokenize
import pickle
import numpy as np

import torch
from solver import TrainSolver

from model import PointerNetworks

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"RE_DIGITS":1,"UNKNOWN":2,"PADDING":0}
        self.word2count = {"RE_DIGITS":1,"UNKNOWN":1,"PADDING":1}
        self.index2word = {0: "PADDING", 1: "RE_DIGITS", 2: "UNKNOWN"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.strip('\n').strip('\r').split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1



def mytokenizer(inS,all_dict):

    #repDig = re.sub(r'\d+[\.,/]?\d+','RE_DIGITS',inS)
    repDig = re.sub(r'\d*[\d,]*\d+', 'RE_DIGITS', inS)
    toked = word_tokenize(repDig)
    or_toked = word_tokenize(inS)
    re_unk_list = []
    ori_list = []

#     print("----------")
#     print(toked)
#     print(or_toked)
#     for i in range(len(toked)):
#         if toked[i] != "RE_DIGITS" and toked[i] != or_toked[i]:
#             print(f"{i} is not matched")
#             print(f"{toked[i]} {or_toked[i]}")
#             break

    for (i,t) in enumerate(toked):
        # print(f"{i}, {t}")
        if t not in all_dict and t not in ['RE_DIGITS']:
            re_unk_list.append('UNKNOWN')
            ori_list.append(or_toked[i])
        else:
            re_unk_list.append(t)
            ori_list.append(or_toked[i])

    labey_edus = [0]*len(re_unk_list)
    labey_edus[-1] = 1




    return ori_list,re_unk_list,labey_edus



def get_mapping(X,Y,D):

    X_map = []
    for w in X:
        if w in D:
            X_map.append(D[w])
        else:
            X_map.append(D['UNKNOWN'])

    X_map = np.array([X_map])
    Y_map = np.array([Y])



    return X_map,Y_map


def print_Time():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S:%m")
    return current_time



def main_input_output(df2):

    #print("start")

    mymodel = PointerNetworks(voca_size =2, voc_embeddings=np.ndarray(shape=(2,300), dtype=float),word_dim=300, hidden_dim=10,is_bi_encoder_rnn=True,rnn_type='GRU',rnn_layers=3,
                 dropout_prob=0.5,use_cuda=False,finedtuning=True,isbanor=True)

    mymodel = torch.load(r'trained_model.torchsave', map_location=lambda storage, loc: storage)
    mymodel.use_cuda = False
    mymodel.eval()
    
    
    mysolver = TrainSolver(mymodel, '', '', '', '', '',
                           batch_size=1, eval_size=1, epoch=10, lr=1e-2, lr_decay_epoch=1, weight_decay=1e-4,
                           use_cuda=False)
    c = 0
    for i in range(len(df['Full Text'])):
        try:
            all_voc = r'all_vocabulary.pickle'
            voca = pickle.load(open(all_voc, 'rb'))
            voca_dict = voca.word2index
            ori_X, X, Y = mytokenizer(df2['Full Text'][i], voca_dict)
            X_in, Y_in = get_mapping(X, Y, voca_dict)

            test_batch_ave_loss, test_pre, test_rec, test_f1, visdata = mysolver.check_accuracy(X_in, Y_in)

            start_b = visdata[3][0]
            end_b = visdata[2][0] + 1
            segments = []

            for j, END in enumerate(end_b):
                START = start_b[j]
                segments.append(' '.join(ori_X[START:END]))
            if(i%500 == 0):
                print(datetime.now())
            #print(segments)
            df2.at[i, 'Segments'] = segments
        #except Exception as e:
        except Exception as e:
            #raise(e)
            #print("skipped at " + str(i))
            pass 
        c += 1
        #if c % 100 == 0:
            #print(c)


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd
from datetime import datetime


df = pd.read_csv('AllReviews0120-1122FullStop.csv')
df["Segments"] = ""
    
if __name__ == '__main__':
    main_input_output(df)
   
    print(print_Time())
    #sent='Sheraton and Pan Am said they are assured under the Soviet joint-venture law that they can repatriate profits from their hotel venture.'
    #output_seg =  main_input_output(sent)
    #print(output_seg)
    #print(print_Time())


2023-03-14 02:33:45.778113
02:33:47:03


In [9]:
df['Segments'][4]

['I am very disappointed with the FitBit Charge 3 .',
 'I had chosen to upgrade in May 2020',
 'to try to exercise more while teaching from home , but am now in the position where I am trying to order my second replacement in less than a year . Just after New Years Day , my Charge 3 stopped charging',
 '( no pun intended )',
 'less than a day after the top part of the screen stopped working . The same occurred in May 2021 , 2 days later after the 1 year anniversary of my original Charge 3 purchase . I have now contacted customer service 3 times in less than a week ,',
 'promised an email every time ,',
 'and have not yet received one .',
 'I am disappointed',
 'and frustrated with my experience ,',
 'particularly since I loved the device',
 'while it was working .']

In [5]:
import pandas as pd
df = pd.read_csv('AllReviews0120-1122.csv')
df["Segments"] = ""
df.head()


,ReviewID,Page Type,Full Text,Rating,Segments
0,1,review,"When they work they are great, however, in my ...",1,
1,2,review,I have a garmin Vivosmart 4 and this fitbit ch...,1,
2,3,review,Absolute Trash! Seriously felt like I bought t...,1,
3,4,review,I got this to pair with my apple watch and cha...,1,
4,5,review,I am very disappointed with the FitBit Charge ...,1,
